In [ ]:
# use ner environment




QA with GPT model tuned with our context

process:

### 1. Create Index for our document in the vector database eg. Pinecone, Chroma

For this we have to generate embeddings of our document. for this we take "text-embedding-ada-002" model.
A batch size is how many embeddings we create and insert(upsert) into vector db at once.


### 2. Retrival of the most relavent chunks.

we can use following command to retrive the relavent chunks.

get relevant contexts (including the questions)

'''res = index.query(xq, top_k=5, include_metadata=True)'''

here top_k = 5 which means top most 5 relavent chunks.

### 3. Answer generation using LLM model eg. GPT-3.5, GPT-3.5-turbo, GPT-4,  

with the retrival is done we have to feed this to LLM model to generate the answers.

For LLM model we have to create a "Prompt". (Here comes "prompt engineers" into picture)

Prompt Parts:

    a. Instruction

    b. Context
    
    c. Query
    
    d. Answer indicator

 



In [ ]:
# ! pip install --upgrade openai

In [ ]:
# ! setx OPENAI_API_KEY "sk-jN0I29A77VHVOF9G49enT3BlbkFJf2Gu9PENkF0AidrSm"

In [ ]:
import openai
import os
OPENAI_API_KEY = "sk-TmN3FyBNwM2X1ok030wLT3BlbkFJpQy0IjDP47lqOZxQm"


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
   return client.embeddings.create(input = text, model=model).data[0].embedding

In [ ]:
s = "this is a test sentence"
emb = get_embedding(s, model='text-embedding-ada-002')
print(emb)
# emb = client.embeddings.create(input=s, engine='text-embedding-ada-002')

In [ ]:
from openai import OpenAI
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
out = completion.choices[0].message
print(out)

In [ ]:
for i in out:
    print(i)
# out

In [ ]:
contentt = out.content
contentt.split('\n')

In [ ]:
# from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [ ]:
response.choices[0].message.content

In [ ]:
response1 = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)

In [ ]:
out2 = response1.choices[0].message.content
print(out2)

In [ ]:
def update_chat(messages, role, content):
  messages.append({"role": role, "content": content})
  return messages

In [ ]:
def get_chatgpt_response(messages):
  response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)
  res = response['choices'][0]['message']['content']
  return  res # return the response message content

In [ ]:
import pprint

messages=[
      {"role": "system", "content": "You are a helpful festival guide."},
      {"role": "user", "content": "I am Nikhil, I want to learn about Diwali festival."},
      {"role": "assistant", "content": "Hello, Nikhil Thats awesome, what do you want to know aboout Diwali festival"},
  ]

while True:
  pprint.pprint(messages)
  user_input = input()
  messages = update_chat(messages, "user", user_input)
  model_response = get_chatgpt_response(messages)
  messages = update_chat(messages, "assistant", model_response)

In [ ]:
# print(openai.__version__) # 1.2.3

## Embedding generate using OpenAI embedding

In [1]:
import os
from openai import OpenAI
OPENAI_API_KEY = "sk-TmN3FyBNwM2X1ok030wLT3BlbkFJpQy0IjDP47lqOZxQm"
client = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
   return client.embeddings.create(input = text, model=model) #.data[0].embedding

In [3]:
text=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ]

In [21]:
res = get_embedding(text)
res

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.0031113410368561745, 0.011778287589550018, -0.005067089572548866, -0.027249081060290337, -0.01629284769296646, 0.032343123108148575, -0.01626589521765709, -0.0010688391048461199, -0.025861022993922234, -0.006677507422864437, 0.020200975239276886, 0.016710612922906876, -0.00916388537734747, 0.02344876527786255, -0.010167869739234447, 0.013449348509311676, 0.025295017287135124, -0.016872329637408257, 0.012061289511620998, -0.01633327826857567, -0.004251773003488779, -0.006434933748096228, -0.004346107132732868, 0.020834362134337425, -0.010578896850347519, -0.003628494217991829, 0.01367170736193657, -0.026319216936826706, -0.0003295376955065876, -0.0021343091502785683, 0.0057981787249445915, -0.010066797025501728, -0.028219375759363174, -0.016211990267038345, -0.004238296765834093, 0.007405227515846491, -0.002927726600319147, -0.03142673522233963, 0.02388000674545765, -0.03328646346926689, -0.00032490520970895886, 0.013112441636621952,

In [32]:
print(f"vector 0: {len(res.data[0].embedding)}\nvector 1: {len(res.data[1].embedding)}")

vector 0: 1536
vector 1: 1536


In [35]:
embeds = [record.embedding for record in res.data]
len(embeds)

2

In [62]:
import pinecone 
from tqdm.autonotebook import tqdm
from langchain.vectorstores import Pinecone
# initialize pinecone
pinecone.init(
    api_key="493b441c-f1fe-44a1-ad4a-0fbb8d469",  # find at app.pinecone.io
    environment="gcp-starter"  # next to api key in console
)

index_name = "forth-qa"
index = pinecone.Index(index_name)

In [38]:
from langchain.document_loaders import DirectoryLoader

directory = 'F:\documents\data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

5


In [40]:
docs

[Document(page_content='The relationship between India and China is complex and multifaceted, shaped by historical interactions, geopolitical dynamics, economic interests, and regional power struggles. These two neighboring countries, each with ancient civilizations and rich cultures, share a relationship that oscillates between cooperation, competition, and occasional tension.\n\nHistorically, India and China have maintained cultural and trade ties dating back centuries through the ancient Silk Road. However, the modern relationship has been marked by territorial disputes, geopolitical competition, and occasional military confrontations, notably the border conflict in 1962 that resulted in a souring of relations between the two nations.', metadata={'source': 'F:\\documents\\data\\india-china.txt'}),
 Document(page_content="One of the primary sources of tension between India and China lies in their unresolved border disputes, particularly in the Himalayan region. Both nations claim ter

In [44]:
docs[0].metadata

{'source': 'F:\\documents\\data\\india-china.txt'}

In [54]:
trec = [{'text': x.page_content} for x in docs]

In [56]:
trec[0]['text']

'The relationship between India and China is complex and multifaceted, shaped by historical interactions, geopolitical dynamics, economic interests, and regional power struggles. These two neighboring countries, each with ancient civilizations and rich cultures, share a relationship that oscillates between cooperation, competition, and occasional tension.\n\nHistorically, India and China have maintained cultural and trade ties dating back centuries through the ancient Silk Road. However, the modern relationship has been marked by territorial disputes, geopolitical competition, and occasional military confrontations, notably the border conflict in 1962 that resulted in a souring of relations between the two nations.'

In [64]:
from tqdm.auto import tqdm
import time
count = 0  # we'll use the count to create unique IDs
batch_size = 32  # process everything in batches of 32
for i in tqdm(range(0, len(trec[0]['text']), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(trec[0]['text']))
    # get batch of lines and IDs
    lines_batch = trec[0]['text'][i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    # res = openai.Embedding.create(input=lines_batch, engine=MODEL)
    res = get_embedding(lines_batch)
    embeds = [record.embedding for record in res.data]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))
    time.sleep(25)

  0%|          | 0/23 [00:00<?, ?it/s]

## Retrieval method

In [ ]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)